In [1]:
import numpy as np
import pytest
import torch
from torch import nn

from typing import Callable, List, Optional, Union

NOTE: this is used as slideshow, and should not to be run.

# RecPack: An Experimental Toolkit for Recommendation Algorithms

by Lien Michiels and Robin Verachtert

## What is Recpack?
- Python library for recommendation algorithm implementation and evaluation

## What is Recpack?
- Python library for recommendation algorithm implementation and evaluation
- Focus on being easily extendable

## What is Recpack?
### Architecture
![pipeline](RecpackPipeline.png)

This image has been designed using resources from Flaticon.com

TODO: include scenarios!

## This Demo
- Demonstrate implementation of new algorithm using "Neural Matrix Factorization (He et al. 2017)"

## This Demo
- Demonstrate implementation of new algorithm using "Neural Matrix Factorization (He et al. 2017)"
- Run experiment to compare performance of new algorithm to baselines

## Neural Matrix Factorization
- Users and items are represented by embedding fectors
- Similarity is modeled using an MLP network, rather than computing <u,i> as in traditional matrix factorization embedding techniques.

### Implementing MLP network

![MLPArchitecture](MLPArchitecture.png)


In [2]:
class MLP(nn.Module):
    """A multi-layer perceptron module.
    This module is a sequence of linear layers plus activation functions.
    The user can optionally add normalization and/or dropout to each of the layers.
    
    Code used from https://github.com/facebookresearch/multimodal/blob/5dec8a/torchmultimodal/modules/layers/mlp.py
    
    :param in_dim: Input dimension.
    :type in_dim: int
    :param out_dim: Output dimension.
    :type out_dim: int
    :param hidden_dims: Output dimension for each hidden layer.
    :type hidden_dims: Optional[Union[int, List[int]]] 
    :param dropout: Probability for dropout layers between each hidden layer.
    :type dropout: float
    :param activation: Which activation function to use. 
        Supports module type or partial.
    :type activation: Callable[..., nn.Module]
    """
    def __init__(
        self, 
        in_dim: int, 
        out_dim: int,
        hidden_dims: Optional[Union[int, List[int]]] = None,
        dropout: float = 0.5,
        activation: Callable[..., nn.Module] = nn.ReLU,
    ):
        super().__init__()

        layers = nn.ModuleList()

        if hidden_dims is None:
            hidden_dims = []

        if isinstance(hidden_dims, int):
            hidden_dims = [hidden_dims]

        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(in_dim, hidden_dim))
            layers.append(activation())
            layers.append(nn.Dropout(dropout))
            in_dim = hidden_dim
        layers.append(nn.Linear(in_dim, out_dim))
        self.model = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

## Implementing the Neural Architecture

![Architecture](NeuMFArchitecture.png)

In [7]:
class NMFModule(nn.Module):
    """Model that encodes the Neural Matrix Factorization Network.
    
    Implements the 3 tiered network defined in the He et al. paper.

    :param predictive_powers: size of the last hidden layer in MLP.
        Embedding sizes computed as 2 * predictive powers.
    :type predictive_powers: int
    :param n_users: number of users in the network
    :type n_users: int
    :param n_items: number of items in the network
    :type n_items: int
    :param hidden_dims: dimensions of the MLP hidden layers.
    :type hidden_dims: Union[int, List[int]]
    :param dropout: Dropout chance between layers of the MLP
    :type dropout: float
    """
    pass

In [8]:
class NMFModule(NMFModule):
    def __init__(
        self, predictive_powers: int, n_users: int, n_items: int, dropout: float
    ):
        super().__init__()
        num_components = 2 * predictive_powers
        
        self.user_embedding = nn.Embedding(n_users, num_components)
        self.item_embedding = nn.Embedding(n_items, num_components)

        # we use a three tiered MLP as described in the experiments of the paper.
        hidden_dims = [
            4 * predictive_powers, 
            2 * predictive_powers, 
            predictive_powers
        ]

        # Output is always 1, since we need a single score for u,i
        self.mlp = MLP(4 * predictive_powers, 1, 
                       hidden_dims, dropout=dropout)

        self.final = nn.Sigmoid()

        # weight initialization
        self.user_embedding.weight.data.normal_(0, 
            1.0 / self.user_embedding.embedding_dim)
        self.item_embedding.weight.data.normal_(0, 
            1.0 / self.item_embedding.embedding_dim)

In [9]:
class NMFModule(NMFModule):
    def forward(self, users: torch.LongTensor, items: torch.LongTensor) -> torch.FloatTensor:
        """Predict scores for the user item pairs obtained 
        by zipping together the two inputs

        :param users: 1D tensor with user ids
        :type users: torch.LongTensor
        :param items: 1D tensor with item ids
        :type items: torch.LongTensor
        :return: 1D tensor with predicted similarities.
            Position i is the similarity between 
            `users[i]` and `items[i]`
        :rtype: torch.FloatTensor
        """

        # Embedding lookups
        user_emb = self.user_embedding(users)
        item_emb = self.item_embedding(items)

        # Pass concatenated through MLP and apply sigmoid
        return self.final(
            self.mlp(
                torch.hstack([user_emb, item_emb])
            )
        )

In [10]:
def test_output_shapes_NMF(
    predictive_factors, num_users, num_items
):
    """Check that no mather the inner settings of the network, the output is always correct"""
    mod = NMFModule(predictive_factors, num_users, num_items, 0.0)
    
    user_tensor = torch.LongTensor([1, 2])
    item_tensor = torch.LongTensor([1, 2])
    
    res = mod(user_tensor, item_tensor) # predict scores for items given the users
    
    assert res.shape == (2, 1)

    assert (res.detach().numpy() <= 1).all()
    assert (res.detach().numpy() >= 0).all()


test_output_shapes_NMF(5, 10, 10)
test_output_shapes_NMF(5, 3, 10)
test_output_shapes_NMF(1, 3, 3)



In [11]:
from typing import List, Union, Optional

import pandas as pd
from recpack.algorithms.base import TorchMLAlgorithm
from recpack.algorithms.samplers import PositiveNegativeSampler
from recpack.algorithms.util import get_users
from recpack.matrix import InteractionMatrix
from scipy.sparse import csr_matrix, lil_matrix


## Implementing the algorithm

### Choosing the right baseclass
`Algorithm`

Follows the sklearn interface
- `__init__(...)`: sets the (hyper)parameters of the algorithm
- `fit(X)`: train the algorithm, and build a model that can be used for prediction
- `predict(X)`: Given a matrix of user histories, recommend items for these users.

### Choosing the right baseclass
![algorithm baseclasses](AlgorithmBaseclasses.png)

### Choosing the right baseclass
`TorchMLAlgorithm`
- `__init__`
    - Adds standard parameters (learning rate, batch_size, num_epochs, keep_last, ...)
- `fit(X, validation_data)`:
    - Epoch loop -> Implement training single epoch
    - Early stopping
    - Keep best/last model
    - Progress logs
- `_predict`
    - Batched prediction loop -> Implement prediction for single batch
    - Prune recommendations 
- `save` + `load`

TODO: Need a better way to indicate what remains to be implemented?

### Choosing the right baseclass
`TorchMLAlgorithm`

We need to implement:

- `__init__`: Add hyperparameters
- `_init_model`: Initialise the model during training
- `_train_epoch`: train for a single epoch
- `_predict_batch`: predict for a batch of users


In [12]:
class NeuMF(TorchMLAlgorithm):
    """Implementation of Neural Matrix Factoration.

    Neural Matrix Factorization based on MLP architecture
    as presented in Figure 2 in He, Xiangnan, et al. 
    "Neural collaborative filtering."
    In Proceedings of the 26th international conference on world wide web. 2017.

    Represents the users and items using an embedding, 
    similarity between the two is modelled using a neural network.

    The network consists of an embedding for both users and items.
    To compute similarity those two embeddings are 
    concatenated and passed through the MLP
    Finally the similarity is transformed to the [0,1] domain
    using a sigmoid function.

    As in the paper, the sum of square errors is used as loss function.
    Positive items should get a prediction close to 1, 
    while sampled negatives should get a value close to 0.

    The MLP has 3 layers, as suggested in the experiments section.
    Bottom layer has dimension `4 * predictive_powers`, 
    middle layer `2 * predictive_powers`
    and the top layer has `predictive_powers`.

    :param predictive_powers: Size of the last hidden layer in the MLP network.
        Embedding size is 2 * predictive_powers
    :type predictive_powers: int
    :param batch_size: How many samples to use in each update step.
        Higher batch sizes make each epoch more efficient,
        but increases the amount of epochs needed to converge to the optimum,
        by reducing the amount of updates per epoch.
        Defaults to 512.
    :type batch_size: Optional[int]
    :param max_epochs: The max number of epochs to train.
        If the stopping criterion uses early stopping, less epochs could be used.
        Defaults to 10.
    :type max_epochs: Optional[int]
    :param learning_rate: How much to update the weights at each update. Defaults to 0.01
    :type learning_rate: Optional[float]
    :param stopping_criterion: Name of the stopping criterion to use for training.
        For available values,
        check :meth:`recpack.algorithms.stopping_criterion.StoppingCriterion.FUNCTIONS`
        Defaults to 'ndcg'
    :type stopping_criterion: Optional[str]
    :param stop_early: If True, early stopping is enabled,
        and after ``max_iter_no_change`` iterations where improvement of loss function
        is below ``min_improvement`` the optimisation is stopped,
        even if max_epochs is not reached.
        Defaults to False
    :type stop_early: bool, optional
    :param max_iter_no_change: If early stopping is enabled,
        stop after this amount of iterations without change.
        Defaults to 5
    :type max_iter_no_change: int, optional
    :param min_improvement: If early stopping is enabled, no change is detected,
        if the improvement is below this value.
        Defaults to 0.01
    :type min_improvement: float, optional
    :param seed: Seed to the randomizers, useful for reproducible results,
        defaults to None
    :type seed: int, optional
    :param save_best_to_file: If true, the best model will be saved after training,
        defaults to False
    :type save_best_to_file: bool, optional
    :param keep_last: Retain last model, rather than best
        (according to stopping criterion value on validation data), defaults to False
    :type keep_last: bool, optional
    :param predict_topK: The topK recommendations to keep per row in the matrix.
        Use when the user x item output matrix would become too large for RAM.
        Defaults to None, which results in no filtering.
    :type predict_topK: int, optional
    :param n_negatives_per_positive: Amount of negatives to sample for each positive example, defaults to 1
    :type n_negatives_per_positive: int, optional
    :param dropout: Dropout parameter used in MLP, defaults to 0.0
    :type dropout: float, optional
    :param exact_sampling: Enable or disable exact checks while sampling. 
        With exact sampling the sampled negatives are guaranteed to not have been visited by the user. 
        Non exact sampling assumes that the space for item selection is large enough, 
        such that most items are likely not seen before.
        Defaults to False,
    :type exact_sampling: bool, optional
    """
    pass

In [13]:
class NeuMF(NeuMF):
    def __init__(
        self,
        predictive_factors: int,
        n_negatives_per_positive: Optional[int] = 1,
        dropout: Optional[float] = 0.0,
        # baseclass parameters 
        # ...
    ):
        super().__init__(batch_size, max_epochs, learning_rate,
            stopping_criterion, stop_early, max_iter_no_change,
            min_improvement, seed, save_best_to_file, keep_last,
            predict_topK,
        )

        self.predictive_factors = predictive_factors

        self.n_negatives_per_positive = n_negatives_per_positive
        self.dropout = dropout

        self.sampler = PositiveNegativeSampler(
            U=self.n_negatives_per_positive, replace=False, 
            batch_size=self.batch_size
        )

In [14]:
class NeuMF(NeuMF):
    def _init_model(self, X: csr_matrix):
        num_users, num_items = X.shape
        self.model_ = NMFModule(
            self.predictive_factors, num_users, num_items, self.dropout
        ).to(self.device)

        self.optimizer = torch.optim.Adam(
            self.model_.parameters(), lr=self.learning_rate
        )

In [15]:
class NeuMF(NeuMF):
    def _train_epoch(self, X: csr_matrix) -> List[int]:
        losses = []
        for users, positives, negatives in self.sampler.sample(X):

            self.optimizer.zero_grad()

            # Predict for the positives
            positive_scores = self.model_(
                users.to(self.device), positives.to(self.device))
            # Predict for the negatives
            negative_scores = self.model_(
                *self._construct_negative_prediction_input(
                    users.to(self.device), negatives.to(self.device))
            )

            loss = self._compute_loss(
                positive_scores, negative_scores)

            # Backwards propagation of the loss
            loss.backward()
            self.optimizer.step()

            losses.append(loss.item())

        return losses



In [16]:
class NeuMF(NeuMF):
    def _compute_loss(
        self, positive_scores: torch.FloatTensor, negative_scores: torch.FloatTensor
    ) -> torch.FloatTensor:
        """Compute the Square Error loss given recommendations 
        for positive items, and sampled negatives.
        """

        mse = nn.MSELoss(reduction="sum")
        return mse(positive_scores, torch.ones_like(positive_scores, dtype=torch.float)) + mse(
            negative_scores, torch.zeros_like(negative_scores, dtype=torch.float)
        )

    def _construct_negative_prediction_input(self, users, negatives):
        """Construct the prediction input given a 1D user tensor and a 2D negatives tensor.
        
        Since negatives has shape |batch| x U, and users is a 1d vector,
        these need to be turned into two 1D vectors of shape |batch| * U

        First the users as a row are stacked U times and transposed,
        so that this is also a batch x U tensor.
        Then both are reshaped to remove the 2nd dimension, 
        resulting in a single long 1d vector.
        """
        return (
            users.repeat(self.n_negatives_per_positive, 1).T.reshape(-1), 
            negatives.reshape(-1)
        )

In [17]:
class NeuMF(NeuMF):
    def _batch_predict(
        self, X: csr_matrix, users: List[int]
    ) -> csr_matrix:
        """Generate recommendations for each of the users."""

        X_pred = lil_matrix(X.shape)
        if users is None:
            users = get_users(X)

        _, n_items = X.shape
        n_users = len(users)

        # Create tensors such that each user, item pair gets a score.
        # The user tensor contains the users in order 
        # (eg. [1, 1, 2, 2]), 
        # item indices are repeated (eg. [0, 1, 2, 0, 1, 2]).
        user_tensor = torch.LongTensor(users).repeat(
            n_items, 1).T.reshape(-1).to(self.device)
        item_tensor = torch.arange(n_items).repeat(
            n_users).to(self.device)

        X_pred[users] = self.model_(
            user_tensor, item_tensor
        ).detach().cpu().numpy().reshape(n_users, n_items)
        return X_pred.tocsr()

## Experiment

Use RecPack Pipeline to compare the newly implemented algorithm to frequently used baselines

In [23]:
from recpack.pipelines import PipelineBuilder
from recpack.datasets import MovieLens25M
from recpack.scenarios import WeakGeneralization

In [24]:
DATASET_PATH = '/home/robinverachtert/datasets'

In [25]:
dataset = MovieLens25M(
    path=DATASET_PATH
)
data = dataset.load_interaction_matrix()

  0%|          | 0/12415224 [00:00<?, ?it/s]

  0%|          | 0/12415224 [00:00<?, ?it/s]

### Choosing the right scenario

- LastItemPrediction
- StrongGeneralization
- StrongGeneralizationTimed
- StrongGeneralizationTimedMostRecent
- Timed
- WeakGeneralization

In [27]:
scenario = WeakGeneralization(frac_data_in=0.8, validation=True)
scenario.split(data)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [28]:
from recpack.pipelines import ALGORITHM_REGISTRY
ALGORITHM_REGISTRY.register('NeuMF', NeuMF)

In [29]:
builder = PipelineBuilder()
builder.set_data_from_scenario(scenario)

builder.add_metric('NDCGK', K=10)
builder.add_metric('CoverageK', K=10)

builder.add_algorithm(
    algorithm = 'NeuMF', 
    params = {
        'batch_size': 128,
        'max_epochs': 10,
        'learning_rate': 0.01,
        'stopping_criterion': 'ndcg',
        'predict_topK': 20,
        'n_negatives_per_positive': 3,
        'dropout': 0.01
    },
    grid = {
        'predictive_factors': [8, 16, 32],
    }
)

builder.add_algorithm('Popularity', params={'K': 20})
builder.add_algorithm(
    'ItemKNN', 
    grid={'similarity': ['conditional_probability', 'cosine']}
)
builder.set_optimisation_metric('NDCGK', K=10)

In [30]:
pipeline = builder.build()

In [31]:
pipeline.run()

  0%|          | 0/3 [00:00<?, ?it/s]

128 10 0.01 ndcg
2022-06-22 19:34:27,322 - base - recpack - INFO - Processed epoch 0 in 245.66 s.Batch Training Loss = 39.9801
2022-06-22 20:03:52,854 - stopping_criterion - recpack - INFO - StoppingCriterion has value 0.0807752474141612, which is better than previous iterations.
2022-06-22 20:03:52,855 - base - recpack - INFO - Model improved. Storing better model.
2022-06-22 20:03:52,876 - base - recpack - INFO - Evaluation at end of 0 took 1765.55 s.
2022-06-22 20:07:56,205 - base - recpack - INFO - Processed epoch 1 in 243.32 s.Batch Training Loss = 33.0413
2022-06-22 20:37:07,710 - stopping_criterion - recpack - INFO - StoppingCriterion has value 0.08761402269236272, which is better than previous iterations.
2022-06-22 20:37:07,711 - base - recpack - INFO - Model improved. Storing better model.
2022-06-22 20:37:07,733 - base - recpack - INFO - Evaluation at end of 1 took 1751.53 s.
2022-06-22 20:41:10,411 - base - recpack - INFO - Processed epoch 2 in 242.67 s.Batch Training Loss 

2022-06-23 07:01:36,645 - base - recpack - INFO - Evaluation at end of 9 took 1416.01 s.
2022-06-23 07:01:36,665 - base - recpack - INFO - Fitting NeuMF complete - Took 2.02e+04s
128 10 0.01 ndcg
2022-06-23 07:40:50,696 - base - recpack - INFO - Processed epoch 0 in 509.58 s.Batch Training Loss = 38.9571
2022-06-23 08:12:34,783 - stopping_criterion - recpack - INFO - StoppingCriterion has value 0.08408753015707665, which is better than previous iterations.
2022-06-23 08:12:34,784 - base - recpack - INFO - Model improved. Storing better model.
2022-06-23 08:12:34,861 - base - recpack - INFO - Evaluation at end of 0 took 1904.16 s.
2022-06-23 08:21:03,305 - base - recpack - INFO - Processed epoch 1 in 508.44 s.Batch Training Loss = 32.6848
2022-06-23 08:48:31,171 - stopping_criterion - recpack - INFO - StoppingCriterion has value 0.0898314980420112, which is better than previous iterations.
2022-06-23 08:48:31,172 - base - recpack - INFO - Model improved. Storing better model.
2022-06-23

2022-06-23 19:37:32,117 - base - recpack - INFO - Evaluation at end of 9 took 1723.12 s.
2022-06-23 19:37:32,134 - base - recpack - INFO - Fitting NeuMF complete - Took 2.1e+04s
2022-06-23 20:06:31,710 - base - recpack - INFO - Fitting Popularity complete - Took 2.55s


/home/robinverachtert/dist-env/lib/python3.8/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


2022-06-23 20:06:58,574 - base - recpack - INFO - Fitting ItemKNN complete - Took 17.5s
2022-06-23 20:07:33,150 - base - recpack - INFO - Fitting ItemKNN complete - Took 15.6s
2022-06-23 20:08:26,023 - base - recpack - INFO - Fitting ItemKNN complete - Took 15.6s


In [32]:
pipeline.get_metrics_dataframe()

,ndcgk_10,coveragek_10
"NeuMF(batch_size=128,dropout=0.01,exact_sampling=False,keep_last=False,learning_rate=0.01,max_epochs=10,max_iter_no_change=5,min_improvement=0.0,n_negatives_per_positive=3,predict_topK=20,predictive_factors=16,save_best_to_file=False,seed=3337940912,stop_early=False,stopping_criterion=<recpack.algorithms.stopping_criterion.StoppingCriterion object at 0x7feb478a0340>)",0.104866,0.122034
Popularity(K=20),0.085165,0.000502
"ItemKNN(K=200,normalize=False,normalize_X=False,normalize_sim=False,pop_discount=None,similarity=cosine)",0.162156,0.147565


In [35]:
pipeline.optimisation_results

,identifier,params,NDCGK
0,"NeuMF(batch_size=128,dropout=0.01,exact_sampli...","{'predictive_factors': 8, 'batch_size': 128, '...",0.083669
1,"NeuMF(batch_size=128,dropout=0.01,exact_sampli...","{'predictive_factors': 16, 'batch_size': 128, ...",0.087949
2,"NeuMF(batch_size=128,dropout=0.01,exact_sampli...","{'predictive_factors': 32, 'batch_size': 128, ...",0.082559
3,"ItemKNN(K=200,normalize=False,normalize_X=Fals...",{'similarity': 'conditional_probability'},0.103835
4,"ItemKNN(K=200,normalize=False,normalize_X=Fals...",{'similarity': 'cosine'},0.138635


In [33]:
pipeline.get_metrics_dataframe(short=True)

,ndcgk_10,coveragek_10
NeuMF,0.104866,0.122034
Popularity,0.085165,0.000502
ItemKNN,0.162156,0.147565


## Check it out!

<div style="text-align: center;">
    <img src="Recpack Demo - QR codes(3).png" alt="codes"/>
</div>

### Reach out to us
<div style="text-align: center;">
    <img src="Recpack Demo - Contact Info(2).png" alt="drawing"/>
</div>